In [ ]:
from typing import List, Dict, Tuple
from dataclasses import dataclass

@dataclass
class ECTPrompt:
    text: str
    company: str
    type: str = "ECT"
    bucket: int = 0
    model: str = "gpt-4o"

class ECTPromptGenerator:
    def __init__(self, batch_size: int = 4):
        self.batch_size = batch_size
        
    def generate_prompts(self, ect: str, company_id: str, html_prompt: str = "") -> List[ECTPrompt]:
        """
        Generate ECT analysis prompts in batches.
        Each batch contains a subset of headings to analyze.
        """
        # Split headings into batches
        heading_batches = [
            list(ECTConfig.HEADINGS.items())[i:i + self.batch_size]
            for i in range(0, len(ECTConfig.HEADINGS), self.batch_size)
        ]
        
        return [
            ECTPrompt(
                text=self._create_prompt_text(batch, ect, html_prompt),
                company=company_id,
                bucket=batch_idx
            )
            for batch_idx, batch in enumerate(heading_batches)
        ]
    
    def _create_prompt_text(self, headings: List[Tuple[str, List[str]]], ect: str, html_prompt: str) -> str:
        """Create a single analysis prompt for a batch of headings."""
        prompt_parts = [
            f"Answer the following {len(headings)} questions using content from the Earnings call text specified below.",
            "Make sure to include all supporting information available in the text without repeating information.",
            f"Make sure the response is able to answer the {len(headings)} questions below {html_prompt}\n"
        ]
        
        # Add heading-specific questions
        for idx, (heading, metrics) in enumerate(headings, 1):
            metrics_str = ", ".join(metrics)
            prompt_parts.append(
                f"{idx}. How is the {metrics_str} of the company? "
                f"Give an in-depth response under the sub-heading: {heading}. "
                "Use up to 7 bullets to formulate the response."
            )
        
        # Add the earnings text
        prompt_parts.append(f"\nThe Earnings call text is as follows: \n{ect}")
        
        return "\n".join(prompt_parts)

# Usage example:
generator = ECTPromptGenerator(batch_size=4)
prompts = generator.generate_prompts(
    ect="Sample earnings call text...",
    company_id="AAPL",
    html_prompt=""
)

# Convert to dict format if needed for compatibility
prompt_dicts = [
    {
        "prompt": p.text,
        "company": p.company,
        "type": p.type,
        "bucket": p.bucket,
        "model": p.model
    }
    for p in prompts
]